In [4]:
import re
import pandas as pd

# Read the CSV
df = pd.read_csv('C:/Users/User/Downloads/Kyoway Datasheets/cleaned_data_kyoway.csv', low_memory=False)

# Function to normalize the '운전 날짜' column from Japanese format to YYYY-MM-DD
def normalize_date(date_str):
    match = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日', date_str)
    if match:
        year, month, day = match.groups()
        return f"{int(year):04d}-{int(month):02d}-{int(day):02d}"
    return None

# Apply normalization to create a new column
df['normalized_date'] = df['운전 날짜'].apply(normalize_date)

# Update 'date-time' using normalized date and time
def build_datetime(row):
    date = row['normalized_date']
    time = str(row['기본 장입 시작 시각']).strip()
    if pd.isna(date) or pd.isna(time):
        return None
    if time.startswith("24:"):
        fixed_time = time.replace("24:", "00:", 1)
        try:
            dt = pd.to_datetime(date) + pd.Timedelta(days=1)
            return f"{dt.date()} {fixed_time}"
        except:
            return None
    else:
        return f"{date} {time}"

df['date-time'] = df.apply(build_datetime, axis=1)

# Save to new CSV
df.to_csv("cleaned_data_kyoway_with_datetime.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_data_kyoway.csv'

In [7]:
import pandas as pd
import re

# --- Step 1: Load both datasets ---
kyoway_df = pd.read_csv('C:/Users/User/Downloads/Kyoway Datasheets/cleaned_data_kyoway.csv', low_memory=False)
weather_df = pd.read_csv('C:/Users/User/Downloads/jma_weather_data_2024-2025.csv')

# --- Step 2: Normalize Japanese-style date in kyoway_df ---
def normalize_date(date_str):
    match = re.search(r'(\d{4})年(\d{1,2})月(\d{1,2})日', str(date_str))
    if match:
        year, month, day = match.groups()
        return f"{int(year):04d}-{int(month):02d}-{int(day):02d}"
    return None

kyoway_df['normalized_date'] = kyoway_df['운전 날짜'].apply(normalize_date)
kyoway_df['clean_time'] = kyoway_df['기본 장입 시작 시각'].astype(str).str.strip()

# --- Step 3: Create unified datetime column, handling 24:00 rollover ---
def build_datetime(row):
    date = row['normalized_date']
    time = row['clean_time']
    if pd.isna(date) or pd.isna(time):
        return None
    if time.startswith("24:"):
        fixed_time = time.replace("24:", "00:", 1)
        dt = pd.to_datetime(date) + pd.Timedelta(days=1)
        return pd.to_datetime(f"{dt.date()} {fixed_time}")
    return pd.to_datetime(f"{date} {time}", errors='coerce')

kyoway_df['date-time'] = kyoway_df.apply(build_datetime, axis=1)
kyoway_df = kyoway_df.dropna(subset=['date-time'])

# --- Step 4: Build datetime in weather_df ---
weather_df['weather_datetime'] = pd.to_datetime(
    weather_df['Date'] + ' ' + weather_df['Time'],
    format='%Y-%m-%d %H:%M', errors='coerce'
)
weather_df = weather_df.dropna(subset=['weather_datetime'])

# --- Clean 'Precipitation (mm)' column in weather_df ---
def clean_precip(val):
    try:
        return float(val)
    except:
        return 0.0

weather_df['Precipitation (mm)'] = weather_df['Precipitation (mm)'].apply(clean_precip)

# --- Step 5: Merge nearest weather record BEFORE each kyoway timestamp ---
kyoway_df.sort_values('date-time', inplace=True)
weather_df.sort_values('weather_datetime', inplace=True)

merged_df = pd.merge_asof(
    kyoway_df,
    weather_df,
    left_on='date-time',
    right_on='weather_datetime',
    direction='backward'
)

# --- Step 6: Save the result ---
merged_df.to_csv("kyoway_with_weather_merged.csv", index=False)


C:\Users\User\AppData\Local\Temp\ipykernel_24824\4134254042.py:6: DtypeWarning: Columns (3,5,6,7,9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_df = pd.read_csv('C:/Users/User/Downloads/jma_weather_data_2024-2025.csv')
